In [9]:
import ROOT as r
import glob
import os
r.EnableImplicitMT()

In [2]:
files_info_b2oc = [
   {"filename": "./Data/00289228_00000001_1.highstats-Small-B2OC-UP.root",   "block": 5, "polarity": "UP"},
   {"filename": "./Data/00289233_00000001_1.highstats-Small-B2OC-DOWN.root", "block": 6, "polarity": "DOWN"},
   {"filename": "./Data/00289239_00000001_1.highstats-Small-B2OC-DOWN.root", "block": 7, "polarity": "DOWN"},
   {"filename": "./Data/00289237_00000001_1.highstats-Small-B2OC-UP.root",   "block": 8, "polarity": "UP"},
]

files_info_b2cc = [
    {"filename": "./Data/00289229_0000000*_1.highstats-Small-B2CC-UP.root",   "block": 5, "polarity": "UP"},
    {"filename": "./Data/00289331_0000000*_1.highstats-Small-B2CC-DOWN.root", "block": 6, "polarity": "DOWN"},
    {"filename": "./Data/00289235_0000000*_1.highstats-Small-B2CC-DOWN.root", "block": 7, "polarity": "DOWN"},
    {"filename": "./Data/00289231_0000000*_1.highstats-Small-B2CC-UP.root",   "block": 8, "polarity": "UP"}, 
]

bad_runs = [303681, 304475, 304487, 304488, 304489, 304490, 304491]

In [ ]:
def split_by_fill(files_info, treename, decay_label, min_entries=20000):
    for info in files_info:
        file_pattern = info["filename"]
        block = info["block"]
        polarity = info["polarity"]

        file_list = glob.glob(file_pattern) if '*' in file_pattern or '?' in file_pattern else [file_pattern]
        # Check for no files found, or single file that does not exist
        if not file_list or (len(file_list) == 1 and not os.path.isfile(file_list[0])):
            print(f"⚠️ No files found for pattern: {file_pattern}")
            continue

        if len(file_list) > 1:
            print(f"Processing as TChain: {file_list}  (Block {block}, {polarity})\n")
            tree = r.TChain(treename)
            for fname in file_list:
                tree.Add(fname)
        else:
            filename = file_list[0]
            print(f"Processing: {filename}  (Block {block}, {polarity})\n")
            file = r.TFile.Open(filename)
            if not file or file.IsZombie():
                print(f"⚠️ Failed to open file: {filename}")
                continue
            tree = file.Get(treename)
            if not tree:
                print(f"Tree {treename} not found in {filename}")
                file.Close()
                continue

        if not tree or tree.GetEntries() == 0:
            print(f"No entries found in files for tree {treename}")
            if len(file_list) == 1 and 'file' in locals() and file:
                file.Close()
            continue

        unique_fills_in_block = set()
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            if int(tree.block) == block and int(tree.run) not in bad_runs and tree.FILL is not None:
                unique_fills_in_block.add(int(tree.FILL))

        sorted_fills = sorted(unique_fills_in_block)
        print(f"Found {len(sorted_fills)} fills in block {block}")

        run_exclude = " && ".join([f"run != {run}" for run in bad_runs])

        for fill_val in sorted_fills:
            cut = f"FILL == {fill_val} && block == {block} && {run_exclude}"

            n_entries = tree.GetEntries(cut)
            if n_entries < min_entries:
                print(f"Skipping FILL {fill_val} — only {n_entries} entries after filtering.")
                continue

            block_dir = os.path.join("Processed_Data", "2024", "Real", decay_label, f"B{block}")
            os.makedirs(block_dir, exist_ok=True)
            outname = os.path.join(block_dir, f"2024_{decay_label}_{polarity}_B{block}_F{fill_val}.root")   
            out_file = r.TFile(outname, "RECREATE")
            new_tree = tree.CopyTree(cut)
            new_tree.Write()
            out_file.Close()
            print(f"✅ Saved: {outname}  ({n_entries} entries)")

        if len(file_list) == 1 and 'file' in locals() and file:
            file.Close()

In [10]:
def clear_blocks(files_info, treename, decay_label):
    for info in files_info:
        file_pattern = info["filename"]
        block = info["block"]
        polarity = info["polarity"]

        file_list = glob.glob(file_pattern) if '*' in file_pattern or '?' in file_pattern else [file_pattern]
        # Check for no files found, or single file that does not exist
        if not file_list or (len(file_list) == 1 and not os.path.isfile(file_list[0])):
            print(f"⚠️ No files found for pattern: {file_pattern}")
            continue

        if len(file_list) > 1:
            print(f"Processing as TChain: {file_list}  (Block {block}, {polarity})\n")
            tree = r.TChain(treename)
            for fname in file_list:
                tree.Add(fname)
        else:
            filename = file_list[0]
            print(f"Processing: {filename}  (Block {block}, {polarity})\n")
            file = r.TFile.Open(filename)
            if not file or file.IsZombie():
                print(f"⚠️ Failed to open file: {filename}")
                continue
            tree = file.Get(treename)
            if not tree:
                print(f"Tree {treename} not found in {filename}")
                file.Close()
                continue

        if not tree or tree.GetEntries() == 0:
            print(f"No entries found in files for tree {treename}")
            if len(file_list) == 1 and 'file' in locals() and file:
                file.Close()
            continue

        run_exclude = " && ".join([f"run != {run}" for run in bad_runs])
        cut = f"block == {block} && {run_exclude}"
        n_entries = tree.GetEntries(cut)
        block_dir = os.path.join("Processed_Data", "2024", "Real", decay_label)
        os.makedirs(block_dir, exist_ok=True)
        outname = os.path.join(block_dir, f"2024_{decay_label}_{polarity}_B{block}.root")   
        out_file = r.TFile(outname, "RECREATE")
        new_tree = tree.CopyTree(cut)
        new_tree.Write()
        out_file.Close()
        print(f"✅ Saved: {outname}  ({n_entries} entries)")

        if len(file_list) == 1 and 'file' in locals() and file:
            file.Close()

In [16]:
split_by_fill(files_info_b2oc, "ST-b2oc", "B2OC")

Processing: ./Data/00289228_00000001_1.highstats-Small-B2OC-UP.root  (Block 5, UP)

Found 25 fills in block 5
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10059.root  (97942 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10061.root  (299599 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10066.root  (336671 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10069.root  (757938 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10070.root  (108898 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10072.root  (732723 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10073.root  (770704 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10074.root  (737512 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10075.root  (194991 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/B5/2024_B2OC_UP_B5_F10077.root  (548795 entries)
✅ Saved: Pr

In [5]:
split_by_fill(files_info_b2cc, "ST-b2cc", "B2CC")

Processing as TChain: ['./Data/00289231_00000002_1.highstats-Small-B2CC-UP.root', './Data/00289231_00000001_1.highstats-Small-B2CC-UP.root']  (Block 8, UP)

Found 11 fills in block 8
Found 11 fills in block 8
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10214.root  (138035 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10214.root  (138035 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10215.root  (131287 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10216.root  (1499211 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10215.root  (131287 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10216.root  (1499211 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10218.root  (4004782 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10218.root  (4004782 entries)
✅ Saved: Processed_Data/2024/Real/B2CC/B8/2024_B2CC_UP_B8_F10219.root  (4759632 ent

In [ ]:
clear_blocks(files_info_b2oc, "ST-b2oc", "B2OC")

✅ Saved: Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B5.root  (12453113 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/2024_B2OC_DOWN_B6.root  (10145158 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/2024_B2OC_DOWN_B7.root  (9379763 entries)
✅ Saved: Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B8.root  (5828363 entries)


In [11]:
clear_blocks(files_info_b2cc, "ST-b2cc", "B2CC")

Processing as TChain: ['./Data/00289229_00000001_1.highstats-Small-B2CC-UP.root', './Data/00289229_00000003_1.highstats-Small-B2CC-UP.root', './Data/00289229_00000002_1.highstats-Small-B2CC-UP.root']  (Block 5, UP)

✅ Saved: Processed_Data/2024/Real/B2CC/2024_B2CC_UP_B5.root  (43327033 entries)
Processing as TChain: ['./Data/00289331_00000001_1.highstats-Small-B2CC-DOWN.root', './Data/00289331_00000002_1.highstats-Small-B2CC-DOWN.root']  (Block 6, DOWN)

✅ Saved: Processed_Data/2024/Real/B2CC/2024_B2CC_DOWN_B6.root  (35834668 entries)
Processing as TChain: ['./Data/00289235_00000001_1.highstats-Small-B2CC-DOWN.root', './Data/00289235_00000002_1.highstats-Small-B2CC-DOWN.root']  (Block 7, DOWN)

✅ Saved: Processed_Data/2024/Real/B2CC/2024_B2CC_DOWN_B7.root  (41085248 entries)
Processing as TChain: ['./Data/00289231_00000002_1.highstats-Small-B2CC-UP.root', './Data/00289231_00000001_1.highstats-Small-B2CC-UP.root']  (Block 8, UP)

✅ Saved: Processed_Data/2024/Real/B2CC/2024_B2CC_UP_B8.ro